**Develop a model that can accurately recognize food items from images and estimate their calorie content, enabling users to track their dietary intake and make informed food choices.**

In [ ]:
!pip install tensorflow
!pip install keras
!pip install numpy
!pip install matplotlib
!pip install scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to the dataset
dataset_path = '/content/drive/MyDrive/food-101'

# Create the image data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load the training and validation data
train_generator = train_datagen.flow_from_directory(
    f'{dataset_path}/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    f'{dataset_path}/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add the classification layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
checkpoint = ModelCheckpoint('food_recognition_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=[early_stopping, checkpoint]
)

In [ ]:
loss, accuracy = model.evaluate(val_generator)
print(f'Validation loss: {loss:.2f}')
print(f'Validation accuracy: {accuracy:.2f}')

In [ ]:
# Create a dictionary mapping food categories to their average calorie content
calorie_dict = {
    'apple': 95,
    'burger': 300,
    'cake': 400,
    # Add more food categories and their average calorie content
}

def estimate_calories(image):
    # Preprocess the image
    image = np.expand_dims(image, axis=0)

    # Make a prediction
    prediction = model.predict(image)

    # Get the predicted food category
    food_category = train_generator.class_indices[np.argmax(prediction)]

    # Look up the calorie content in the dictionary
    if food_category in calorie_dict:
        return calorie_dict[food_category]
    else:
        return 'Unknown'